## Chapter 13: Naive Bayes

A social network isn’t much good if people can’t network. We have a popular feature that allows members to send messages to other members. And while most members are responsible, a few miscreants persistently spam others, who have begun to complain, so VP of Messaging
has asked you to figure out a way to filter out these spam messages

### A Really Dumb Spam Filter

Imagine a “universe” that consists of receiving a message chosen randomly from all possible messages. Let `S` = event “the message is spam”, `V` = event “the message contains the word 'viagra'.” 

Then Bayes’s Theorem tells us that the probability the message = spam conditional on containing the word 'viagra' is: `P(S|V) = [P(V|S)P(S)] / [P(V|S)P(S) + P(V|Sc)P(Sc)]`

Numerator = probability a message = spam *and* contains viagra, while denominator = probability a message contains 'viagra'. Hence you can think of this calculation as simply representing the proportion of 'viagra' messages that *are* spam.

If we have a large collection of messages we *know* = spam, + a large collection of
messages we *know* are *not* spam, we can easily estimate `P(V|Sc)` and `P(V|Sc)`. If we further assume any message is *equally likely* to be spam or not (so that `P(S) = P(Sc) = .5`), then: `P(S|V) = P(V|S) / [P(V|S) + P(V|Sc)]`

For example, if 50% of spam messages have the word 'viagra', but only 1% of nonspam
messages do, the probability any given viagra-containing email is spam is `.5/(.5+.01) = .98`= 98%.

### A More Sophisticated Spam Filter

Imagine now we have a vocab of *many* words, `{w1, ..., wn}`. To move this into the realm of probability theory, write `Xi` = event “a message contains word `w_i`." Also imagine that (through some unspecified-at-this-point process) we’ve come up w/ an estimate `P(Xi|S)` for probability a spam message contains the ith word, + a similar estimate `P(Xi|Sc)` for probability a *non*spam message contains the ith word.

**Key to Naive Bayes** = making the (**big**) assumption that the presences (or absences) of each word = independent of one another, conditional on a message being spam or not. Intuitively, this assumption means that knowing whether a certain spam message contains
the word “viagra” gives you NO info about if that same message contains the word “rolex.” In math terms, this means that: `P(X1 = x_1, ..., Xn = x_n | S] = P(X1 = x_1 | S) * ... * P(Xn = x_n | S)`

This = an *extreme* assumption. (hence “naive”). Imagine our vocab consists only of the words “viagra” and “rolex,” + that 1/2 of all spam messages are for “cheap viagra” + the other 1/2 are for “authentic rolex.” In this case, the **Naive Bayes estimate** that a spam message contains *both* “viagra” and “rolex” is: `P(X1 = x_1, X2 = x_2 | S] = P(X1 = x_1 | S) * P(X2 = x_2 | S) = .5*.5=.25` = 25%, since we’ve assumed away the knowledge that “viagra” and “rolex” actually never occur together (unrealistic) 

Despite the unrealisticness of this assumption, this model often performs well + is used in actual spam filters. The same Bayes’s Theorem reasoning we used for our “viagra-only” spam filter tells us we can calculate probability a message is spam using the equation: `P(S|X=x) = P(X=x|S) / [P(X=x|S) + P(X=x|Sc)]`

**The Naive Bayes assumption** allows us to compute each probability on the RHS by multiplying together the *individual* probability estimates for each vocab word. In practice = usually want to avoid multiplying lots of probabilities together, to avoid a
problem = **underflow** = CPU's don’t deal well w/ floating-point #'s that're too close to 0. Recalling from algebra that `log(ab) = log(a) + log(b)` and that `exp(log(x)) = x`, we usually compute `{p_1 * ... * p_n}` as the equivalent (but floating-point-friendlier):
`exp(log(p_1) + ... + log(p_n))`. 

The only challenge left is coming up w/ estimates for `P(X_i|S)` and `P(X_i|Sc)`, the
probabilities a spam or nonspam message contains the word `w_i`. If we have a fair number of “training” messages labeled as spam + not-spam, an obvious 1st try = estimate simply as the fraction of spam messages containing word `w_i`.

This causes a big problem == Imagine that in our training set, the vocab word “data” only occurs in nonspam messages. Then we’d estimate `P("data"|S) = `. The result is that our Naive Bayes classifier would *always* assign spam probability 0 to any message containing the word “data,” *even a message like “data on cheap viagra + authentic rolex watches.”* 

To avoid this problem, we usually use some kind of **smoothing**. In particular, choose a **pseudocount** — `k` — + estimate probability of seeing the ith word in a spam as: `P(X_i|S) = (k + # of spams containing w_i)/(2k + # of spams)`, w/ something similar for `P(X_i|Sc)`. That is, **when computing spam probabilities for the ith word, assume we also saw `k` additional spams containing the word + `k` additional spams not containing the word.**

Ex: If “data” occurs in 0/98 spam documents, + if `k` = 1, estimate 1/100 = 0.01, which allows our classifier to still assign some nonzero spam probability to messages that contain the word “data.”

### Implementation

Now we have all the pieces we need to build our classifier. 1st, create a simple function to **tokenize** messages into distinct words by 1st converting each message to lowercase, use `re.findall()` to extract “words” consisting of letters, numbers, + apostrophes, + finally use `set()` to get just distinct words:

In [1]:
def tokenize(message):
    import re
    message = message.lower()
    all_words = re.findall("[a-z0-9]+",message) # find all words
    return set(all_words) # get just unique words

2nd function = count the words in a labeled training set of messages + return a `dictionary` whose keys = words + values = 2-element lists `[spam_count, non_spam_count]`, corresponding to how many times we saw that word in both spam + nonspam messages:

In [2]:
def count_words(train):
    """Training set consits of pairs (message, is_spam)"""
    from collections import defaultdict
    
    counts = defaultdict(lambda: [0,0])
    
    for message, is_spam in train:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] += 1
    return counts

Next = turn counts into estimated probabilities using the smoothing described before. The function returns a list of triplets containing each word, probability of seeing that word in a spam message, + probability of seeing that word in a nonspam message:

In [24]:
def word_probs(counts,total_spams,total_nonspams,k=.5):
    """Turn the word_counts into a list of triplets = [w,p(w|spam),p(w|nonspam)]"""
    return [(w,
           # smoothing
           (spam + k)/(total_spams + 2*k),
           (non_spam + k) / (total_nonspams + 2*k))
           for w, (spam, non_spam) in counts.items()]

Last piece = to use these word probabilities + our Naive Bayes assumptions to assign probabilities to messages:

In [25]:
def spam_prob(word_probs,message):
    import math
    
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0
    
    # iterate through each word in vocabulary
    for word, prob_if_spam, prob_if_nonspam in word_probs:
        #if 'word' appears in message, add log prob of seeing it
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_nonspam)
            
        # if 'word' doesn't appear in message, add log prob of NOT 
        # seeing it == log(1-prob of seeing it)
        else:
            log_prob_if_spam += math.log(1 - prob_if_spam)
            log_prob_if_not_spam += math.log(1 - prob_if_nonspam)
    
    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_nonspam = math.exp(log_prob_if_not_spam)
    
    return prob_if_spam/(prob_if_spam+prob_if_nonspam)

Can put this all together into our Naive Bayes Classifier:

In [26]:
class NaiveBayesClassify:
    
    def __init__(self,k=.5):
        self.k = k
        self.word_probs = []
        
    def train(self,training):
        # count spam + nonspam messages
        num_spams = len([is_spam for message, is_spam in training if is_spam])
        num_nonspams = len(training)- num_spams
        
        # run training  data through pipeline
        word_counts = count_words(training)
        self.word_probs = word_probs(word_counts, num_spams, num_nonspams, self.k)
        
    def classify(self, message):
        return spam_prob(self.word_probs, message)

### Testing Our Model

A good (if somewhat old) data set = SpamAssassin public corpus. We’ll look @ files prefixed w/ `20021010`. (On Windows, might need a program like 7-Zip to decompress + extract them.)

After extracting the data, we should have 3 folders: `spam`, `easy_ham`, `hard_ham`. Each folder contains many emails, each contained in a single file. To keep things really simple, we’ll just look @ subject lines of each email. How do we ID the subject line? Looking through the files, they all seem to start w/ `“Subject:”`. So we’ll look for that:

In [36]:
import glob, re

# modify the path with wherever you've put the files
path = r"C:\spam\*\*"
data = []

## glob.glob returns every filename that matches the wildcarded path
# for each file in our path (folder --> subfolder --> files)
for filename in glob.glob(path):
    # if the word 'harm' is not in the file name, then we have spam
    is_spam = "ham" not in filename
    
    # open each file, search through lines until we find subject line
    with open(filename,'r') as file:
        for line in file:
            if line.startswith("Subject:"):
                # remove the leading "Subject: " and keep what's left
                subject = re.sub(r"^Subject: ", "", line).strip()
                data.append((subject, is_spam))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 3082: character maps to <undefined>

Now we can split the data into training + test data + then build a classifier:

In [27]:
import random

random.seed(0) # just so you get the same answers as me

def split_data(data, prob):
    """split data into fractions [prob, 1 - prob]"""
    results = [], []
    for row in data:
        results[0 if random.random() < prob else 1].append(row)
    return results

train_data, test_data = split_data(data, 0.75)

# create instance of NaiveBayesClassify class
classifier = NaiveBayesClassify()

# use NaiveBayesClassify .train() method
classifier.train(train_data)

And then we can check how our model does:

In [39]:
# triplets = (subject, actual is_spam, predicted spam probability)
classified = [(subject, is_spam, classifier.classify(subject))
              for subject, is_spam in test_data]

from collections import Counter
# assume spam_probability > 0.5 corresponds to spam prediction
# + count the combinations of (actual is_spam, predicted is_spam)
counts = Counter((is_spam, spam_probability > 0.5)
                 for _, is_spam, spam_probability in classified)

print(counts)

Counter({(False, False): 737, (True, False): 25, (True, True): 4})


This gives 101 true positives (spam classified as “spam”), 33 false positives (ham
classified as “spam”), 704 true negatives (ham classified as “ham”), and 38 false negatives
(spam classified as “ham”). This means our precision is 101 / (101 + 33) = 75%, and our
recall is 101 / (101 + 38) = 73%, which are not bad numbers for such a simple model.
It’s also interesting to look at the most misclassified: